In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy


###PLOT_EVENTS###
def plot_events(test_struct, fig, first, times, widthspec, width, fontspec, fontsize):
    f = plt.figure()
    axs = gca

    if (len(args) < 3){
        first=min(test_struct.RLOESSSmoothedVelocities.Time(1), test_struct.OBDVelocity(1).Time(1))
    }
    if (len(args) < 4){
        times = test_struct.RLOESSSmoothedVelocities.Time
    }
    if (len(args) < 6){
        widthspec = 'LineWidth'
        width = 1
    }
    if (len(args) < 8){
        fontspec = 'FontSize'
        fontsize = 7
    }
    
    if (first - 1000 > axs.XLim(1) )
        # plot relative times
        isRelative = True
    else
        isRelative = False
        
    #for i in range (1, length(test_struct.Events) + 1):
    for tmpCell in test_struct.Events
        #need this if using other for loop
        #tmpCell = test_struct.Events{i} 
        xs = [tmpCell{2}(1) tmpCell{2}(1)]
        if( xs(2) < times(end) )
            if( isRelative )
                xs = xs-first
            ys = [axs.YLim(1) axs.YLim(2)]
            line(xs,ys,widthspec,width)
            text(xs(2)+1,ys(2)-1,tmpCell{1}, fontspec, fontsize)


#####PLOT_VELOCITIES####
def plot_velocities(structure):
    #pass these in as params or declare at beginning?
    smoothVel = structure.RLOESSSmoothedVelocities
    catVehicleIndex = structure.CatVehicleIndex
    letter = structure.ExperimentLetter

    
    vehicles = range(1, structure.NumVeh)
    first = smoothVel.Time(1);
    velStdPlot = figure;

    setp(gcf, 'Units', 'centimeters')
    setp(gcf, 'PaperPosition', [0 0 16 5])
    setp(gcf, 'PaperSize', [16 5])

    f = plt.figure()
    
    plt.plot(smoothVel.Time-first, smoothVel.Data(:,catVehicleIndex), '-r', 'LineWidth', 1)
    
    plt.plot(smoothVel.Time-first, smoothVel.Data(:,1:size(smoothVel.Data, 2)), 'color', [0.7 0.7 0.7])
    
    plt.plot(smoothVel.Time-first, smoothVel.Data(:,catVehicleIndex), '-r', 'LineWidth', 1)
    
    #events
    plot_events(structure, gcf, first)
    
    #find mean and standard deviations
    velMean, velStd = plot_all_std(structure, 1:catVehicleIndex, gcf)
    
    throughput = structure.NumVeh / 260 * velMean * 3600
    
    plt.xlabel('Time (s)')
    plt.ylabel('Velocity (m/s)')
    plt.title('Velocities with standard deviations for Experiment ' letter)
    
    #save figure
    plt.savefig('experiment_' letter '_std')
    
    plt.show()
    
    
####PLOT_TRAJECTORIES####
def plot_trajectories(structure):
    displacements = structure.RLOESSSmoothedDisplacements
    letter = structure.ExperimentLetter
    
    L = 260 #course length
    dataM = np.remainder(displacements.Data, L)
    diffs = np.diff(dataM.T.ravel)
    toNan = np.argwhere(diffs < -10) #indices where diffs < -10
    dataM(toNan) = NaN
    
    first = structure.RLOESSSmoothedVelocities.Time(1);
    times = displacements.Time-first;
    
    trajFig = figure('Position', [100, 100, 1200, 400])
    setp(trajFig, 'Units', 'centimeters')
    
    f = plt.figure
    
    for i in range(0, displacements.Data(2) + 1) #for i=0 to i = length of displacements.Data(2)
        #DataM(:, i) is the ith column of DataM
        plt.plot(times, DataM(:,i), '-', 'color', [0.7 0.7 0.7])
        
    plt.plot(times, DataM(:, structure.CatVehicleIndex), 'r-')
    plt.plot_events(structure, gcf, first)
    
    plt.xlabel('Time (s)')
    plt.ylabel('Position on road (m)')
    plt.title('Vehicle trajectories for Experiment ' letter)
    
    #save figure
    plt.savefig('experiment_' letter '_traj')
    
    plt.show()
    
    
####PLOT_FUEL_CONSUMPTION####
def plot_fuel_consumption(structure):
    events = structure.Events
    velocities = structure.RLOESSSmoothedVelocities
    obdLitersPerHr = structure.OBDLitersPerHr
    obdVelocity = structure.OBDVelocity
    
    first = velocities.Time(1)
    
    fuelConsumptFig = figure('Position', [100, 100, 1200, 200])
    setp(fuelConsumptFig, 'Units', 'Centimeters')
    
    plt.xlabel('Time (s)')
    plt.ylabel('Fuel (L/100km)')
    plt.title(' ')
    
    consumption = []
    
    #compute total fuel used
    for (testPeriod in range(0, length(events))):
        for (vehNum in range(0, size(velocities, 2)))
            #use 1 instead of 2 bc matlab is 1-based and python is 0-based
            periodStart = np.array(events{testPeriod}(1))
            periodEnd = np.array(events{testPeriod + 1}(1))
            
            #argmax finds first true occurrence
            firstPt = np.argmax(obdLitersPerHr(vehNum).Time>=periodStart)
            lastPt = np.argmax(obdLitersPerHr(vehNum).Time>=periodEnd)
            
            #compute total fuel used by all cars in each period
            totalFuel(testPeriod) = totalFuel(testPeriod) + \
            (1/3600)*sum(obdLitersPerHr(vehNum).Data((firstPt+1):lastPt) \
            .*(obdLitersPerHr(vehNum).Time((firstPt+1):lastPt) \
            -obdLitersPerHr(vehNum).Time(firstPt:(lastPt-1))));
            
            #compute total distance traveled by all cars in each period
            totalDistance(testPeriod) = totalDistance(testPeriod) + \
            (1/3600)*sum(obdVelocity(vehNum).Data((firstPt+1):lastPt) \
            .*(obdVelocity(vehNum).Time((firstPt+1):lastPt) \
            -obdVelocity(vehNum).Time(firstPt:(lastPt-1))));
            
        figure(fuelConsumptFig)
        plt.plot([periodStart-first, periodEnd-first], [totalFuel(testPeriod)/totalDistance(testPeriod)*100, totalFuel(testPeriod)/totalDistance(testPeriod)*100], '--r', 'LineWidth', 1)
        consumption = [consumption, totalFuel(testPeriod)/totalDistance(testPeriod)*100]
    
    plot_events(structure, fuelConsumptFig, first)
    
    #title --sprintf?
    #plt.title(sprintf('Average fuel consumption per period for all vehicles, Experiment %1d', test_struct.Number)
    plt.saveFig('experiment_' structure.ExperimentLetter '_fuelConsumption')
    
    plt.show()